In [1]:
import os
os.environ["OPENAI_API_KEY"] = '你的OpenAI API Key'

# 1.Load 导入Document Loaders
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import Docx2txtLoader
from langchain.document_loaders import TextLoader

# 加载Documents
documents = []
for file in os.listdir('OneFlower'): 
    if file.endswith('.pdf'):
        pdf_path = './OneFlower/' + file
        loader = PyPDFLoader(pdf_path)
        documents.extend(loader.load())
    elif file.endswith('.docx') or file.endswith('.doc'):
        doc_path = './OneFlower/' + file
        loader = Docx2txtLoader(doc_path)
        documents.extend(loader.load())
    elif file.endswith('.txt'):
        text_path = './OneFlower/' + file
        loader = TextLoader(text_path)
        documents.extend(loader.load())

In [2]:
# 2.Split 将Documents切分成块以便后续进行嵌入和向量存储
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, 
                                               chunk_overlap=10)
chunked_documents = text_splitter.split_documents(documents)

In [3]:
# 3.Store 将分割嵌入并存储在矢量数据库Qdrant中
from langchain.vectorstores import Qdrant
from langchain.embeddings import OpenAIEmbeddings
vectorstore = Qdrant.from_documents(
    documents=chunked_documents, # 以分块的文档
    embedding=OpenAIEmbeddings(), # 用OpenAI的Embedding Model做嵌入
    location=":memory:",  # in-memory 存储
    collection_name="my_documents",) # 指定collection_name

In [4]:
# 4. Retrieval 准备模型和Retrieval链
import logging # 导入Logging工具
from langchain.chat_models import ChatOpenAI # ChatOpenAI模型
from langchain.retrievers.multi_query import MultiQueryRetriever # MultiQueryRetriever工具
from langchain.chains import RetrievalQA # RetrievalQA链

# 设置Logging
logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)

# 实例化一个大模型工具 - OpenAI的GPT-3.5
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# 实例化一个MultiQueryRetriever
retriever_from_llm = MultiQueryRetriever.from_llm(retriever=vectorstore.as_retriever(), llm=llm)

# 实例化一个RetrievalQA链
qa_chain = RetrievalQA.from_chain_type(llm,retriever=vectorstore.as_retriever())

In [9]:
# 5. 问答展示
def ask_question(query):
    # 使用RetrievalQA链来获取答案
    response = qa_chain(query)
    
    # 返回得到的答案
    return response

# 为用户提供交互界面进行问答
while True:
    # 获取用户的问题
    user_query = input("请随意提问 (或者输入'exit'退出): ")

    # 如果用户输入"exit"，则退出循环
    if user_query.lower() == 'exit':
        break

    # 使用定义的函数获取答案，并打印
    answer = ask_question(user_query)
    print("答案:", answer)

print("谢谢使用 QA 系统!")

请随意提问 (或者输入'exit'退出):  易速鲜花的董事长致辞，总结一下


答案: {'query': '易速鲜花的董事长致辞，总结一下', 'result': '董事长在致辞中表示热烈欢迎新员工加入易速鲜花集团，并为公司能拥有优秀员工感到自豪和骄傲。他对员工即将为公司付出的努力表示诚挚的感谢，并表示易速鲜花将会因员工的努力工作而稳步健康发展，因员工的贡献而更加精彩。'}


请随意提问 (或者输入'exit'退出):  董事长致辞中的企业精神具体是什么？


答案: {'query': '董事长致辞中的企业精神具体是什么？', 'result': '企业精神在董事长致辞中被描述为"团结向上，完美无缺"。这意味着易速鲜花希望每一位员工都能以高度的主人翁责任感和使命感，积极发扬团结合作的精神，在各自的岗位上勤奋敬业，尽职尽责，奋力拼搏。同时，易速鲜花也强调员工之间应该默契配合，相互接纳，取长补短，共同奋进，同舟共济。这样，易速鲜花可以成为一个坚强的战斗堡垒，在发展的道路上克服困难，迎接挑战。'}


请随意提问 (或者输入'exit'退出):  谢谢


答案: {'query': '谢谢', 'result': '您好，欢迎光临易速鲜花！请问您需要帮助吗？'}


请随意提问 (或者输入'exit'退出):  exit


谢谢使用 QA 系统!
